In [44]:
import json
from pprint import pprint
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Image
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from IPython.display import SVG
from keras.models import Model, model_from_json, load_model
from keras.layers import Input, Dense, LSTM, concatenate, Reshape
from keras.utils.vis_utils import plot_model, model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam, SGD

In [55]:
gold_fl_nm = "/Users/jaydeep/jaydeep_workstation/ASU/Research/oaei/2018/KnowledgeGraphs/anatomy-dataset/ref_json.json"
with open(gold_fl_nm) as f:
    gold_data = json.load(f)
    
gold_copy = {}
for g in gold_data['@graph']:
    gold_copy[g['entity']] = g['actual']

In [4]:
model_nm = '/Users/jaydeep/jaydeep_workstation/ASU/Research/oaei/my_model.h5'
model = load_model(model_nm)

In [11]:
train_fl_nm = '/Users/jaydeep/jaydeep_workstation/ASU/Research/oaei/source_data.json'
with open(train_fl_nm) as f:
    train_data = json.load(f)
    train_data_X = []
    train_data_Y = []
    for key in train_data.keys():
        hierarchies = train_data[key]['hierarchy']
        for hierarchy in hierarchies:
            tmp = []
            words = hierarchy.split('|')
            for word in words:
                if word in train_data:
                    tmp.append(train_data[word]['vector'])
            train_data_X.append(tmp)
            train_data_Y.append(train_data[key]['vector'])

In [6]:
test_fl_nm = '/Users/jaydeep/jaydeep_workstation/ASU/Research/oaei/target_data.json'
with open(test_fl_nm) as f:
    test_data = json.load(f)

In [7]:
for key in test_data.keys():
    hierarchies = test_data[key]['hierarchy']
    for hierarchy in hierarchies:
        test_data_X = []
        tmp = []
        words = hierarchy.split('|')
        for word in words:
            if word in test_data:
                tmp.append(test_data[word]['vector'])
#         test_data_X.append(tmp)\\n\",\n",
        t_x = np.array(tmp)
        t_x = t_x.reshape(1, t_x.shape[0], t_x.shape[1])
        pred = model.predict(t_x);
        test_data[key]['simConcept'] = pred.tolist()

with open('target_data_output.json', 'w') as outfile:
    json.dump(test_data, outfile, indent=1)

In [8]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [58]:
import sys
test_op_fl_nm = '/Users/jaydeep/jaydeep_workstation/ASU/Research/oaei/target_data_output.json'
with open(test_op_fl_nm) as f:
    test_op_data = json.load(f)

op_copy = {}
    
for test_key in test_op_data.keys():
    cosine_val = 0
    output = ''
    pred_op = test_op_data[test_key]['simConcept']
    for train_key in train_data.keys():
        u = train_data[train_key]['vector']
        v = pred_op[0]
        cos_sim = cosine(u,v)
        if(cos_sim>cosine_val):
            cosine_val = cos_sim
            key = test_key
            val = train_key
    op_copy[key] = val

In [59]:
tp = 0;
fp = 0;
for gold in gold_copy:
    if(gold_copy[gold] == op_copy[gold]):
        tp = tp + 1
    else:
        fp = fp + 1
        
pprint(tp)
pprint(fp)

0
1497
